# Gemini API: Customer Support Agent

### Install Packages

In [ ]:
!pip install -U -q google-generativeai==0.7.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.9/163.9 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 718.3/718.3 kB 6.7 MB/s eta 0:00:00


### Import packages

Import the necessary packages.

In [ ]:
import pathlib
import textwrap
import time

import google.generativeai as genai
import google.ai.generativelanguage as glm

from IPython import display
from IPython.display import Markdown

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

### Set up your API key

Before you can use the Gemini API, you must first obtain an API key. If you don't already have one, create a key with one click in Google AI Studio.

<a class="button button-primary" href="https://makersuite.google.com/app/apikey" target="_blank" rel="noopener noreferrer">Get an API key</a>


In [ ]:
try:
    # Used to securely store your API key
    from google.colab import userdata

    # Or use `os.getenv('API_KEY')` to fetch an environment variable.
    GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
except ImportError:
    import os
    GOOGLE_API_KEY = os.environ['GOOGLE_API_KEY']

genai.configure(api_key=GOOGLE_API_KEY)

### Customer Support Chatbot

In [ ]:
def get_order_status(order_id: str) -> str:
    """Fetches the status of a given order ID."""
    # Mock data for example purposes
    order_statuses = {
        "12345": "Shipped",
        "67890": "Processing",
        "11223": "Delivered"
    }
    return order_statuses.get(order_id, "Order ID not found.")

def initiate_return(order_id: str, reason: str) -> str:
    """Initiates a return for a given order ID with a specified reason."""
    # Mock data for example purposes
    if order_id in ["12345", "67890", "11223"]:
        return f"Return initiated for order {order_id} due to: {reason}."
    else:
        return "Order ID not found. Cannot initiate return."

### Create Gemini Client

In [ ]:
model = genai.GenerativeModel(
    model_name='gemini-1.5-flash-latest',
    tools=[get_order_status, initiate_return] # list of all available tools
)

In [ ]:
model._tools.to_proto()

[function_declarations {
   name: "get_order_status"
   description: "Fetches the status of a given order ID."
   parameters {
     type_: OBJECT
     properties {
       key: "order_id"
       value {
         type_: STRING
       }
     }
     required: "order_id"
   }
 }
 function_declarations {
   name: "initiate_return"
   description: "Initiates a return for a given order ID with a specified reason."
   parameters {
     type_: OBJECT
     properties {
       key: "order_id"
       value {
         type_: STRING
       }
     }
     properties {
       key: "reason"
       value {
         type_: STRING
       }
     }
     required: "order_id"
     required: "reason"
   }
 }]

### alway use the model in chat mode for function calling

In [ ]:
chat = model.start_chat(enable_automatic_function_calling=True)

While this was all handled automatically, if you need more control, you can:

- Leave the default `enable_automatic_function_calling=False` and process the `glm.FunctionCall` responses yourself.
- Or use `GenerativeModel.generate_content`, where you also need to manage the chat history.

In [ ]:
response = chat.send_message('What is the status of order 67890?')
response.text

'The order is currently being processed. \n'

In [ ]:
response.candidates

[content {
  parts {
    text: "The order is currently being processed. \n"
  }
  role: "model"
}
finish_reason: STOP
index: 0
safety_ratings {
  category: HARM_CATEGORY_SEXUALLY_EXPLICIT
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_HATE_SPEECH
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_HARASSMENT
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_DANGEROUS_CONTENT
  probability: NEGLIGIBLE
}
]

Examine the chat history to see the flow of the conversation and how function calls are integrated within it.

The `ChatSession.history` property stores a chronological record of the conversation between the user and the Gemini model. Each turn in the conversation is represented by a [`glm.Content`](https://ai.google.dev/api/python/google/ai/generativelanguage/Content) object, which contains the following information:

*   **Role**: Identifies whether the content originated from the "user" or the "model".
*   **Parts**: A list of [`glm.Part`](https://ai.google.dev/api/python/google/ai/generativelanguage/Part) objects that represent individual components of the message. With a text-only model, these parts can be:
    *   **Text**: Plain text messages.
    *   **Function Call** ([`glm.FunctionCall`](https://ai.google.dev/api/python/google/ai/generativelanguage/FunctionCall)): A request from the model to execute a specific function with provided arguments.
    *   **Function Response** ([`glm.FunctionResponse`](https://ai.google.dev/api/python/google/ai/generativelanguage/FunctionResponse)): The result returned by the user after executing the requested function.

 In the previous example with the mittens calculation, the history shows the following sequence:

1.  **User**: Asks the question about the total number of mittens.
1.  **Model**: Determines that the multiply function is helpful and sends a FunctionCall request to the user.
1.  **User**: The `ChatSession` automatically executes the function (due to `enable_automatic_function_calling` being set) and sends back a `FunctionResponse` with the calculated result.
1.  **Model**: Uses the function's output to formulate the final answer and presents it as a text response.

In [ ]:
for content in chat.history:
    part = content.parts[0]
    print(content.role, "->", type(part).to_dict(part))
    print('-'*80)

user -> {'text': 'What is the status of order 67890?'}
--------------------------------------------------------------------------------
model -> {'function_call': {'name': 'get_order_status', 'args': {'order_id': '67890'}}}
--------------------------------------------------------------------------------
user -> {'function_response': {'name': 'get_order_status', 'response': {'result': 'Processing'}}}
--------------------------------------------------------------------------------
model -> {'text': 'The order is currently being processed. \n'}
--------------------------------------------------------------------------------


In [ ]:
response = chat.send_message('I want to return order 11223 because it is defective.')
print(response.text)

OK. I have initiated a return for order 11223. 



In [ ]:
response.candidates

[content {
  parts {
    text: "OK. I have initiated a return for order 11223. \n"
  }
  role: "model"
}
finish_reason: STOP
index: 0
safety_ratings {
  category: HARM_CATEGORY_SEXUALLY_EXPLICIT
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_HATE_SPEECH
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_HARASSMENT
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_DANGEROUS_CONTENT
  probability: NEGLIGIBLE
}
]

In [ ]:
for content in chat.history:
    part = content.parts[0]
    print(content.role, "->", type(part).to_dict(part))
    print('-'*80)

user -> {'text': 'What is the status of order 67890?'}
--------------------------------------------------------------------------------
model -> {'function_call': {'name': 'get_order_status', 'args': {'order_id': '67890'}}}
--------------------------------------------------------------------------------
user -> {'function_response': {'name': 'get_order_status', 'response': {'result': 'Processing'}}}
--------------------------------------------------------------------------------
model -> {'text': 'The order is currently being processed. \n'}
--------------------------------------------------------------------------------
user -> {'text': 'I want to return order 11223 because it is defective.'}
--------------------------------------------------------------------------------
model -> {'function_call': {'name': 'initiate_return', 'args': {'order_id': '11223', 'reason': 'defective'}}}
--------------------------------------------------------------------------------
user -> {'function_respon

### Sequential Function Calls or Nested Calls

Output of the first function call becomes the input to the second!

    order_statuses = {
        "12345": "Shipped",
        "67890": "Processing",
        "11223": "Delivered"
    }

In [ ]:
response = chat.send_message('Can you check the status of order 11223? If its delivered, please initiat return as it was the wrong order')
print(response.text)

Order 11223 has been delivered. I have initiated a return because it was the wrong order. 



In [ ]:
for content in chat.history:
    part = content.parts[0]
    print(content.role, "->", type(part).to_dict(part))
    print('-'*80)

user -> {'text': 'What is the status of order 67890?'}
--------------------------------------------------------------------------------
model -> {'function_call': {'name': 'get_order_status', 'args': {'order_id': '67890'}}}
--------------------------------------------------------------------------------
user -> {'function_response': {'name': 'get_order_status', 'response': {'result': 'Processing'}}}
--------------------------------------------------------------------------------
model -> {'text': 'The order is currently being processed. \n'}
--------------------------------------------------------------------------------
user -> {'text': 'I want to return order 11223 because it is defective.'}
--------------------------------------------------------------------------------
model -> {'function_call': {'name': 'initiate_return', 'args': {'order_id': '11223', 'reason': 'defective'}}}
--------------------------------------------------------------------------------
user -> {'function_respon

# Sequential function calls - 2

    order_statuses = {
        "12345": "Shipped",
        "67890": "Processing",
        "11223": "Delivered"
    }

In [ ]:
response = chat.send_message('Can you check the status of order 67890? If its delivered, please initiat return as it was the wrong order')
print(response.text)

Order 67890 is still being processed. I can initiate a return once it's delivered. 



In [ ]:
response = chat.send_message('Can you check the status of order 67890? If its delivered, please initiat return as it was the wrong order else cancel the order.')
print(response.text)

Order 67890 is still being processed. I cannot initiate a return as it is not delivered yet.  Do you want me to cancel the order instead? 



In [ ]:
for content in chat.history:
    part = content.parts[0]
    print(content.role, "->", type(part).to_dict(part))
    print('-'*80)

user -> {'text': 'What is the status of order 67890?'}
--------------------------------------------------------------------------------
model -> {'function_call': {'name': 'get_order_status', 'args': {'order_id': '67890'}}}
--------------------------------------------------------------------------------
user -> {'function_response': {'name': 'get_order_status', 'response': {'result': 'Processing'}}}
--------------------------------------------------------------------------------
model -> {'text': 'The order is currently being processed. \n'}
--------------------------------------------------------------------------------
user -> {'text': 'I want to return order 11223 because it is defective.'}
--------------------------------------------------------------------------------
model -> {'function_call': {'name': 'initiate_return', 'args': {'order_id': '11223', 'reason': 'defective'}}}
--------------------------------------------------------------------------------
user -> {'function_respon

### Adding a funciton to handle cancel

In [ ]:
def get_order_status(order_id: str) -> str:
    """Fetches the status of a given order ID."""
    # Mock data for example purposes
    order_statuses = {
        "12345": "Shipped",
        "67890": "Processing",
        "11223": "Delivered"
    }
    return order_statuses.get(order_id, "Order ID not found.")

def initiate_return(order_id: str, reason: str) -> str:
    """Initiates a return for a given order ID with a specified reason."""
    # Mock data for example purposes
    if order_id in ["12345", "67890", "11223"]:
        return f"Return initiated for order {order_id} due to: {reason}."
    else:
        return "Order ID not found. Cannot initiate return."

def cancel_order(order_id: str) -> str:
    """Cancels a given order ID if possible."""
    # Mock data for example purposes
    order_statuses = {
        "12345": "Shipped",
        "67890": "Processing",
        "11223": "Delivered"
    }
    if order_id in order_statuses:
        if order_statuses[order_id] == "Processing":
            return f"Order {order_id} has been canceled successfully."
        else:
            return f"Order {order_id} cannot be canceled as it is already {order_statuses[order_id]}."
    else:
        return "Order ID not found. Cannot cancel order."

add cancel to the list of functions.

In [ ]:
model = genai.GenerativeModel(
    model_name='gemini-1.5-flash-latest',
    tools=[get_order_status, initiate_return, cancel_order]
)

In [ ]:
chat = model.start_chat(enable_automatic_function_calling=True)

In [ ]:
response = chat.send_message('Can you check the status of order 67890? If its delivered, please initiat return as it was the wrong order else cancel the order.')
print(response.text)

OK. I have canceled order 67890. 



In [ ]:
for content in chat.history:
    part = content.parts[0]
    print(content.role, "->", type(part).to_dict(part))
    print('-'*80)

user -> {'text': 'Can you check the status of order 67890? If its delivered, please initiat return as it was the wrong order else cancel the order.'}
--------------------------------------------------------------------------------
model -> {'function_call': {'name': 'get_order_status', 'args': {'order_id': '67890'}}}
--------------------------------------------------------------------------------
user -> {'function_response': {'name': 'get_order_status', 'response': {'result': 'Processing'}}}
--------------------------------------------------------------------------------
model -> {'function_call': {'name': 'cancel_order', 'args': {'order_id': '67890'}}}
--------------------------------------------------------------------------------
user -> {'function_response': {'name': 'cancel_order', 'response': {'result': 'Order 67890 has been canceled successfully.'}}}
--------------------------------------------------------------------------------
model -> {'text': 'OK. I have canceled order 678

### Parallel Function Calling. Can it handle those?

    order_statuses = {
        "12345": "Shipped",
        "67890": "Processing",
        "11223": "Delivered"
    }

In [ ]:
model = genai.GenerativeModel(
    model_name='gemini-1.5-flash-latest',
    tools=[get_order_status, initiate_return, cancel_order]
)

chat = model.start_chat(enable_automatic_function_calling=True)

In [ ]:
response = chat.send_message("What is the status of order 12345? Also, can you cancel order 67890 and initiate a return for order 11223 because it is defective?")

In [ ]:
response.text

'The status of order 12345 is Shipped. Order 67890 has been canceled successfully. A return has been initiated for order 11223 because it is defective. \n'

### How many functions it can handle

In [ ]:
def get_order_status(order_id: str) -> str:
    """Fetches the status of a given order ID."""
    # Mock data for example purposes
    order_statuses = {
        "12345": "Shipped",
        "67890": "Processing",
        "11223": "Delivered"
    }
    return order_statuses.get(order_id, "Order ID not found.")

def initiate_return(order_id: str, reason: str) -> str:
    """Initiates a return for a given order ID with a specified reason."""
    # Mock data for example purposes
    if order_id in ["12345", "67890", "11223"]:
        return f"Return initiated for order {order_id} due to: {reason}."
    else:
        return "Order ID not found. Cannot initiate return."

def cancel_order(order_id: str) -> str:
    """Cancels a given order ID if possible."""
    # Mock data for example purposes
    order_statuses = {
        "12345": "Shipped",
        "67890": "Processing",
        "11223": "Delivered"
    }
    if order_id in order_statuses:
        if order_statuses[order_id] == "Processing":
            return f"Order {order_id} has been canceled successfully."
        else:
            return f"Order {order_id} cannot be canceled as it is already {order_statuses[order_id]}."
    else:
        return "Order ID not found. Cannot cancel order."

def update_shipping_address(order_id: str, new_address: str) -> str:
    """Updates the shipping address for a given order ID."""
    # Mock data for example purposes
    if order_id in ["12345", "67890", "11223"]:
        return f"Shipping address for order {order_id} has been updated to: {new_address}."
    else:
        return "Order ID not found. Cannot update shipping address."

def track_shipment(tracking_number: str) -> str:
    """Tracks the shipment with the given tracking number."""
    # Mock data for example purposes
    tracking_info = {
        "TRACK123": "In Transit",
        "TRACK456": "Delivered",
        "TRACK789": "Out for Delivery"
    }
    return tracking_info.get(tracking_number, "Tracking number not found.")

def apply_discount(order_id: str, discount_code: str) -> str:
    """Applies a discount to the given order ID."""
    # Mock data for example purposes
    valid_discount_codes = ["DISCOUNT10", "SAVE20"]
    if order_id in ["12345", "67890", "11223"]:
        if discount_code in valid_discount_codes:
            return f"Discount code {discount_code} applied to order {order_id}."
        else:
            return f"Invalid discount code: {discount_code}."
    else:
        return "Order ID not found. Cannot apply discount."

def change_payment_method(order_id: str, payment_method: str) -> str:
    """Changes the payment method for a given order ID."""
    # Mock data for example purposes
    if order_id in ["12345", "67890", "11223"]:
        return f"Payment method for order {order_id} has been changed to: {payment_method}."
    else:
        return "Order ID not found. Cannot change payment method."

def provide_invoice(order_id: str) -> str:
    """Provides an invoice for the given order ID."""
    # Mock data for example purposes
    if order_id in ["12345", "67890", "11223"]:
        return f"Invoice for order {order_id} has been sent to your email."
    else:
        return "Order ID not found. Cannot provide invoice."

def extend_warranty(order_id: str, years: int) -> str:
    """Extends the warranty for a given order ID."""
    # Mock data for example purposes
    if order_id in ["12345", "67890", "11223"]:
        return f"Warranty for order {order_id} has been extended by {years} years."
    else:
        return "Order ID not found. Cannot extend warranty."

def check_product_availability(product_id: str) -> str:
    """Checks the availability of a product with the given product ID."""
    # Mock data for example purposes
    product_availability = {
        "PROD123": "In Stock",
        "PROD456": "Out of Stock",
        "PROD789": "Limited Stock"
    }
    return product_availability.get(product_id, "Product ID not found.")

In [ ]:
model = genai.GenerativeModel(
    model_name='gemini-1.5-flash-latest',
    tools=[
        get_order_status, initiate_return, cancel_order, update_shipping_address,
        track_shipment, apply_discount, change_payment_method, provide_invoice,
        extend_warranty, check_product_availability
    ]
)

In [ ]:
chat = model.start_chat()

In [ ]:
response = chat.send_message("What is the status of order 12345? Can you update the address to 123 Main St, Anytown USA?")
# response.text

In [ ]:
response.candidates

[content {
  parts {
    function_call {
      name: "get_order_status"
      args {
        fields {
          key: "order_id"
          value {
            string_value: "12345"
          }
        }
      }
    }
  }
  parts {
    function_call {
      name: "update_shipping_address"
      args {
        fields {
          key: "new_address"
          value {
            string_value: "123 Main St, Anytown USA"
          }
        }
        fields {
          key: "order_id"
          value {
            string_value: "12345"
          }
        }
      }
    }
  }
  role: "model"
}
finish_reason: STOP
index: 0
safety_ratings {
  category: HARM_CATEGORY_HARASSMENT
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_HATE_SPEECH
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_SEXUALLY_EXPLICIT
  probability: NEGLIGIBLE
}
safety_ratings {
  category: HARM_CATEGORY_DANGEROUS_CONTENT
  probability: NEGLIGIBLE
}
]

In [ ]:
# Print out each of the function calls requested from this single call.
for part in response.parts:
    if fn := part.function_call:
        args = ", ".join(f"{key}={val}" for key, val in fn.args.items())
        print(f"{fn.name}({args})")

get_order_status(order_id=12345)
update_shipping_address(new_address=123 Main St, Anytown USA, order_id=12345)


In [ ]:
responses = {
    'get_order_status': get_order_status(order_id="12345"),
    'update_shipping_address': update_shipping_address(order_id="12345", new_address="123 Main St, Anytown USA")
}

In [ ]:
get_order_status(order_id="12345")

'Shipped'

In [ ]:
# Build the response parts.
response_parts = [
    glm.Part(function_response=glm.FunctionResponse(name=fn, response={"result": val}))
    for fn, val in responses.items()
]

responses = chat.send_message(response_parts)
print(responses.text)

The order is currently shipped. I've updated the shipping address to 123 Main St, Anytown USA. 



In [ ]:
chat = model.start_chat(enable_automatic_function_calling=True)

In [ ]:
# response = chat.send_message("What product available in your stock?")
response.text

'Please provide a product ID. \n'

In [ ]:
response = chat.send_message("PROD789 product available in your stock?")
response.text

'PROD789 is available in limited stock. \n'

In [ ]:
for content in chat.history:
    part = content.parts[0]
    print(content.role, "->", type(part).to_dict(part))
    print('-'*80)

user -> {'text': 'What product available in your stock?'}
--------------------------------------------------------------------------------
model -> {'function_call': {'name': 'check_product_availability', 'args': {'product_id': ''}}}
--------------------------------------------------------------------------------
user -> {'function_response': {'name': 'check_product_availability', 'response': {'result': 'Product ID not found.'}}}
--------------------------------------------------------------------------------
model -> {'text': 'Please provide a product ID. \n'}
--------------------------------------------------------------------------------
user -> {'text': 'PROD789 product available in your stock?'}
--------------------------------------------------------------------------------
model -> {'function_call': {'name': 'check_product_availability', 'args': {'product_id': 'PROD789'}}}
--------------------------------------------------------------------------------
user -> {'function_respons

## Restaurant Chatbot - V1

In [ ]:
import random

# Menu items with prices, reviews, and variations
menu = {
    "juice": {"price": 50, "reviews": "Refreshing and sweet.", "variations": ["small", "medium", "large"]},
    "coffee": {"price": 80, "reviews": "Strong and energizing.", "variations": ["small", "medium", "large"]},
    "rice": {"price": 100, "reviews": "Soft and well-cooked.", "variations": ["half", "full"]},
    "biriyani": {"price": 150, "reviews": "Rich and flavorful.", "variations": ["half", "full"]},
    "chicken": {"price": 200, "reviews": "Tender and juicy.", "variations": ["half", "full"]},
    "beef": {"price": 250, "reviews": "Savory and well-seasoned.", "variations": ["half", "full"]}
}

# Order statuses and data
orders = {}
complaints = {}
discount_codes = {"SAVE10": 10, "SAVE20": 20}

# Function to check the menu
def check_menu() -> str:
    items = ", ".join(menu.keys())
    return f"The available menu items are: {items}."

# Function to get the price of an item
def get_price(item: str) -> str:
    if item in menu:
        return f"The price of {item} is {menu[item]['price']} BDT."
    else:
        return f"{item.capitalize()} is not available in the menu."

# Function to get delivery charge
def get_delivery_charge() -> str:
    return "The delivery charge is 30 BDT."

# Function to get delivery duration
def get_delivery_duration() -> str:
    return "The average delivery duration is 45 minutes."

# Function to get reviews on a specific menu item
def get_reviews(item: str) -> str:
    if item in menu:
        return f"Reviews for {item}: {menu[item]['reviews']}"
    else:
        return f"{item.capitalize()} is not available in the menu."

# Function to place an order
def place_order(items: dict, address: str, payment_mode: str) -> str:
    order_id = str(random.randint(10000, 99999))
    orders[order_id] = {"items": items, "status": "Processing", "address": address, "payment_mode": payment_mode}
    return f"Your order has been placed successfully. Your order ID is {order_id}."

# Function to cancel an order
def cancel_order(order_id: str) -> str:
    if order_id in orders:
        if orders[order_id]["status"] == "Processing":
            orders[order_id]["status"] = "Cancelled"
            return f"Order {order_id} has been canceled successfully."
        else:
            return f"Order {order_id} cannot be canceled as it is already {orders[order_id]['status']}."
    else:
        return "Order ID not found. Cannot cancel order."

# Function to check order status
def check_order_status(order_id: str) -> str:
    if order_id in orders:
        return f"The status of order {order_id} is {orders[order_id]['status']}."
    else:
        return "Order ID not found."

# Function to change payment method
def change_payment_method(order_id: str, payment_method: str) -> str:
    if order_id in orders:
        orders[order_id]["payment_mode"] = payment_method
        return f"Payment method for order {order_id} has been changed to {payment_method}."
    else:
        return "Order ID not found. Cannot change payment method."

# Function to apply discount code
def apply_discount(order_id: str, discount_code: str) -> str:
    if order_id in orders:
        if discount_code in discount_codes:
            discount = discount_codes[discount_code]
            total_price = sum(menu[item]["price"] for item in orders[order_id]["items"])
            discounted_price = total_price - (total_price * discount / 100)
            return f"Discount code {discount_code} applied. New total is {discounted_price} BDT."
        else:
            return "Invalid discount code."
    else:
        return "Order ID not found. Cannot apply discount."

# Function to provide an online payment link
def provide_online_payment_link(order_id: str) -> str:
    if order_id in orders:
        return f"Please pay using the following link: https://paymentgateway.com/pay/{order_id}"
    else:
        return "Order ID not found."

# Function to manage complaints
def manage_complaint(order_id: str, complaint: str) -> str:
    if order_id in orders:
        complaint_id = str(random.randint(10000, 99999))
        complaints[complaint_id] = {"order_id": order_id, "complaint": complaint, "status": "Received"}
        return f"Your complaint has been received. Your complaint ID is {complaint_id}."
    else:
        return "Order ID not found. Cannot lodge complaint."

# Function to check complaint status
def check_complaint_status(complaint_id: str) -> str:
    if complaint_id in complaints:
        return f"The status of complaint {complaint_id} is {complaints[complaint_id]['status']}."
    else:
        return "Complaint ID not found."

# Function to share available offers
def share_offers() -> str:
    return "Current offers: Use SAVE10 for 10% off, SAVE20 for 20% off."

# Function to update stock
def update_stock(item: str, quantity: int) -> str:
    if item in menu:
        menu[item]['stock'] = quantity
        return f"Stock for {item} updated to {quantity} units."
    else:
        return f"{item.capitalize()} is not available in the menu."

# Function to handle item variations
def get_variations(item: str) -> str:
    if item in menu:
        variations = ", ".join(menu[item]["variations"])
        return f"Available variations for {item} are: {variations}."
    else:
        return f"{item.capitalize()} is not available in the menu."

# Function to ask for quantity and variation when placing an order
def ask_order_details(item: str, variation: str, quantity: int) -> str:
    if item in menu:
        if variation in menu[item]["variations"]:
            return f"You have ordered {quantity} {variation} {item}(s)."
        else:
            return f"{variation} is not available for {item}. Available variations are: {', '.join(menu[item]['variations'])}."
    else:
        return f"{item.capitalize()} is not available in the menu."

# Initialize the model with the defined functions
model = genai.GenerativeModel(
    model_name='gemini-1.5-flash-latest',
    tools=[
        check_menu, get_price, get_delivery_charge, get_delivery_duration, get_reviews,
        place_order, cancel_order, check_order_status, change_payment_method, apply_discount,
        provide_online_payment_link, manage_complaint, check_complaint_status, share_offers,
        update_stock, get_variations, ask_order_details
    ]
)

# Start a chat session with the model
chat = model.start_chat(enable_automatic_function_calling=True)

# Example of sending a message to the model
response = chat.send_message("What is the price of biriyani?")
print(response.text)


The price of biriyani is 150 BDT. 



In [ ]:
while True:
  query = input("user query: ")
  response = chat.send_message(query)
  print(response.text)

  if query == "exit":
    break

user query: what the delivery charge?
The delivery charge is 30 BDT. 

user query: ktokkn lgbe aste?
The average delivery duration is 45 minutes. 

user query: ami order krte cai
Ok. What would you like to order? Please tell me the items, quantity and variations (if any). 

user query: biriyani
How many biriyani would you like to order? 

user query: 3packet
We have two variations for biriyani: half and full. Which one would you prefer? 

user query: full
Ok. So that's 3 full biriyani. Please confirm your order: 

* 3 full biriyani

Please confirm if this is correct. 

user query: tumader ki juice ase?
Yes, we have juice. What kind of juice are you interested in? 

user query: ki ki ase?
We have small, medium and large sizes for juice. 



user query: medium size er dao
How many medium size juice would you like to order? 

user query: same 3ta
Ok, so that's 3 full biriyani and 3 medium size juice.  Please confirm your order: 

* 3 full biriyani
* 3 medium size juice

Please confirm if 

In [ ]:
for content in chat.history:
    part = content.parts[0]
    print(content.role, "->", type(part).to_dict(part))
    print('-'*80)

user -> {'text': 'What is the price of biriyani?'}
--------------------------------------------------------------------------------
model -> {'function_call': {'name': 'get_price', 'args': {'item': 'biriyani'}}}
--------------------------------------------------------------------------------
user -> {'function_response': {'name': 'get_price', 'response': {'result': 'The price of biriyani is 150 BDT.'}}}
--------------------------------------------------------------------------------
model -> {'text': 'The price of biriyani is 150 BDT. \n'}
--------------------------------------------------------------------------------
user -> {'text': 'what the delivery charge?'}
--------------------------------------------------------------------------------
model -> {'function_call': {'name': 'get_delivery_charge', 'args': {}}}
--------------------------------------------------------------------------------
user -> {'function_response': {'name': 'get_delivery_charge', 'response': {'result': 'The del

## Restaurant Chatbot - V2

In [ ]:
import random

# Menu items with prices, reviews, and variations
menu = {
    "juice": {"price": 50, "reviews": "Refreshing and sweet.", "variations": ["small", "medium", "large"]},
    "coffee": {"price": 80, "reviews": "Strong and energizing.", "variations": ["small", "medium", "large"]},
    "rice": {"price": 100, "reviews": "Soft and well-cooked.", "variations": ["half", "full"]},
    "biriyani": {"price": 150, "reviews": "Rich and flavorful.", "variations": ["half", "full"]},
    "chicken": {"price": 200, "reviews": "Tender and juicy.", "variations": ["half", "full"]},
    "beef": {"price": 250, "reviews": "Savory and well-seasoned.", "variations": ["half", "full"]}
}

# Order statuses and data
orders = {}
complaints = {}
discount_codes = {"SAVE10": 10, "SAVE20": 20}


# Function to check the menu
def check_menu() -> str:
    """
    Returns a string listing all available menu items.
    """
    items = ", ".join(menu.keys())
    return f"The available menu items are: {items}."

# Function to get the price of an item
def get_price(item: str) -> str:
    """
    Returns the price of a specific item.
    """
    if item in menu:
        return f"The price of {item} is {menu[item]['price']} BDT."
    else:
        return f"{item.capitalize()} is not available in the menu."

# Function to get delivery charge
def get_delivery_charge() -> str:
    """
    Returns the delivery charge.
    """
    return "The delivery charge is 30 BDT."

# Function to get delivery duration
def get_delivery_duration() -> str:
    """
    Returns the average delivery duration.
    """
    return "The average delivery duration is 45 minutes."

# Function to get reviews on a specific menu item
def get_reviews(item: str) -> str:
    """
    Returns reviews for a specific menu item.
    """
    if item in menu:
        return f"Reviews for {item}: {menu[item]['reviews']}"
    else:
        return f"{item.capitalize()} is not available in the menu."

# Function to suggest upsell items
def suggest_upsell(item: str) -> str:
    """
    Suggests complementary items based on the given item.
    """
    suggestions = {
        "biriyani": ["chicken", "beef"],
        "coffee": ["juice"],
        "rice": ["chicken", "beef"]
    }
    if item in suggestions:
        upsell_items = ", ".join(suggestions[item])
        return f"Customers often buy {upsell_items} with {item}. Would you like to add any of these?"
    else:
        return f"No suggestions available for {item}."

# Function to place an order
def place_order(items: dict, address: str, payment_mode: str) -> str:
    """
    Places an order with the specified items, address, and payment mode.
    """
    order_id = str(random.randint(10000, 99999))
    orders[order_id] = {"items": items, "status": "Processing", "address": address, "payment_mode": payment_mode}
    return f"Your order has been placed successfully. Your order ID is {order_id}."

# Function to review the order before final placement
def review_order(items: dict) -> str:
    """
    Returns a summary of the order for review before final placement.
    """
    review = "You have ordered the following items:\n"
    for item, details in items.items():
        review += f"{details['quantity']} x {details['variation']} {item} at {menu[item]['price']} BDT each\n"
    return review + "Would you like to add any more items or proceed to payment?"

# Function to finalize the order after review
def finalize_order(order_id: str, address: str, payment_mode: str) -> str:
    """
    Finalizes the order after review with the given address and payment mode.
    """
    if order_id in orders:
        orders[order_id]["address"] = address
        orders[order_id]["payment_mode"] = payment_mode
        return f"Order {order_id} has been finalized. Payment method: {payment_mode}. Delivery address: {address}."
    else:
        return "Order ID not found. Cannot finalize order."

# Function to cancel an order
def cancel_order(order_id: str) -> str:
    """
    Cancels an order if it is still in processing.
    """
    if order_id in orders:
        if orders[order_id]["status"] == "Processing":
            orders[order_id]["status"] = "Cancelled"
            return f"Order {order_id} has been canceled successfully."
        else:
            return f"Order {order_id} cannot be canceled as it is already {orders[order_id]['status']}."
    else:
        return "Order ID not found. Cannot cancel order."

# Function to check order status
def check_order_status(order_id: str) -> str:
    """
    Returns the status of a specific order.
    """
    if order_id in orders:
        return f"The status of order {order_id} is {orders[order_id]['status']}."
    else:
        return "Order ID not found."

# Function to change payment method
def change_payment_method(order_id: str, payment_method: str) -> str:
    """
    Changes the payment method for a specific order.
    """
    if order_id in orders:
        orders[order_id]["payment_mode"] = payment_method
        return f"Payment method for order {order_id} has been changed to {payment_method}."
    else:
        return "Order ID not found. Cannot change payment method."

# Function to apply discount code
def apply_discount(order_id: str, discount_code: str) -> str:
    """
    Applies a discount code to a specific order.
    """
    if order_id in orders:
        if discount_code in discount_codes:
            discount = discount_codes[discount_code]
            total_price = sum(menu[item]["price"] for item in orders[order_id]["items"])
            discounted_price = total_price - (total_price * discount / 100)
            return f"Discount code {discount_code} applied. New total is {discounted_price} BDT."
        else:
            return "Invalid discount code."
    else:
        return "Order ID not found. Cannot apply discount."

# Function to provide an online payment link
def provide_online_payment_link(order_id: str, payment_method: str) -> str:
    """
    Provides an online payment link for specific payment methods.
    """
    if order_id in orders:
        if payment_method in ["bkash", "nagad"]:
            return f"Please pay using the following link: https://paymentgateway.com/pay/{order_id}?method={payment_method}"
        else:
            return f"Payment method {payment_method} does not require an online link."
    else:
        return "Order ID not found."

# Function to manage complaints
def manage_complaint(order_id: str, complaint: str) -> str:
    """
    Manages a complaint for a specific order.
    """
    if order_id in orders:
        complaint_id = str(random.randint(10000, 99999))
        complaints[complaint_id] = {"order_id": order_id, "complaint": complaint, "status": "Received"}
        return f"Your complaint has been received. Your complaint ID is {complaint_id}."
    else:
        return "Order ID not found. Cannot lodge complaint."

# Function to check complaint status
def check_complaint_status(complaint_id: str) -> str:
    """
    Checks the status of a specific complaint.
    """
    if complaint_id in complaints:
        return f"The status of complaint {complaint_id} is {complaints[complaint_id]['status']}."
    else:
        return "Complaint ID not found."

# Function to share available offers
def share_offers() -> str:
    """
    Shares current promotional offers.
    """
    return "Current offers: Use SAVE10 for 10% off, SAVE20 for 20% off."

# Function to update stock
def update_stock(item: str, quantity: int) -> str:
    """
    Updates the stock quantity for a specific item.
    """
    if item in menu:
        menu[item]['stock'] = quantity
        return f"Stock for {item} updated to {quantity} units."
    else:
        return f"{item.capitalize()} is not available in the menu."

# Function to handle item variations
def get_variations(item: str) -> str:
    """
    Returns available variations for a specific item.
    """
    if item in menu:
        variations = ", ".join(menu[item]["variations"])
        return f"Available variations for {item} are: {variations}."
    else:
        return f"{item.capitalize()} is not available in the menu."

# Function to ask for quantity and variation when placing an order
def ask_order_details(item: str, variation: str, quantity: int) -> str:
    """
    Asks for the quantity and variation of an item when placing an order.
    """
    if item in menu:
        if variation in menu[item]["variations"]:
            return f"You have ordered {quantity} {variation} {item}(s)."
        else:
            return f"{variation} is not available for {item}. Available variations are: {', '.join(menu[item]['variations'])}."
    else:
        return f"{item.capitalize()} is not available in the menu."

# Initialize the model with the defined functions
model = genai.GenerativeModel(
    model_name='gemini-1.5-flash-latest',
    # system_instruction='''You are a restaurant chatbot developed by Kalim.
    #                       Your primary role is to assist customers with various queries and tasks related to the restaurant.
    #                       You are designed to handle menu inquiries, order placements, payment processing, delivery details,
    #                       complaints, and promotional offers. Your goal is to ensure a smooth and satisfying customer
    #                       experience by providing accurate information, suggesting complementary items, and resolving issues efficiently.

    #                       You can handle multiple languages. User query may combined with some local accent with short form even.
    #                       Your responses should be always in the languages the user asking you.

    #                       Instructions for Clarifying Questions:

    #                       Always ask clarifying questions if the user's query is ambiguous or lacks sufficient information to complete the request.
    #                       For example:
    #                       - If a user asks about an item without specifying the variation or quantity, ask for these details.
    #                       - If a user wants to place an order but does not provide a delivery address or payment method, request this information.


    #                       # USING TOOLS YOU ARE CAPABLE OF FOLLOWING:
    #                       # - You are knowledgeable about the restaurant's menu, including item prices, variations, and current promotions that will be.
    #                       # - You can handle multiple variations of queries and provide suggestions to enhance customer satisfaction.
    #                       # - You can process and manage orders, handle payments, address complaints, and provide delivery information.
    #                       ''',


    tool_config = {
      "function_calling_config": {
        "mode": "ANY",
        "allowed_function_names": ["check_menu", "get_price", "get_delivery_charge", "get_delivery_duration", "get_reviews",
                                  "place_order", "finalize_order", "cancel_order", "check_order_status", "change_payment_method", "apply_discount",
                                  "provide_online_payment_link", "manage_complaint", "check_complaint_status", "share_offers",
                                  "update_stock", "get_variations", "ask_order_details", "review_order", "suggest_upsell"
                                  ]
      },
    },
    tools=[
        check_menu, get_price, get_delivery_charge, get_delivery_duration, get_reviews,
        place_order, finalize_order, cancel_order, check_order_status, change_payment_method, apply_discount,
        provide_online_payment_link, manage_complaint, check_complaint_status, share_offers,
        update_stock, get_variations, ask_order_details, review_order, suggest_upsell
    ],
)

# Start a chat session with the model
chat = model.start_chat(enable_automatic_function_calling=True)


In [ ]:
model._tool_config

function_calling_config {
  mode: ANY
  allowed_function_names: "check_menu"
  allowed_function_names: "get_price"
  allowed_function_names: "get_delivery_charge"
  allowed_function_names: "get_delivery_duration"
  allowed_function_names: "get_reviews"
  allowed_function_names: "place_order"
  allowed_function_names: "finalize_order"
  allowed_function_names: "cancel_order"
  allowed_function_names: "check_order_status"
  allowed_function_names: "change_payment_method"
  allowed_function_names: "apply_discount"
  allowed_function_names: "provide_online_payment_link"
  allowed_function_names: "manage_complaint"
  allowed_function_names: "check_complaint_status"
  allowed_function_names: "share_offers"
  allowed_function_names: "update_stock"
  allowed_function_names: "get_variations"
  allowed_function_names: "ask_order_details"
  allowed_function_names: "review_order"
  allowed_function_names: "suggest_upsell"
}

In [ ]:
while True:
  query = input("user query: ")
  response = chat.send_message(query)
  print(response.text)

  if query == "exit":
    break

user query: what item available?


BadRequest: 400 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash-latest:generateContent?%24alt=json%3Benum-encoding%3Dint: Request contains an invalid argument.

In [ ]:
for content in chat.history:
    part = content.parts[0]
    print(content.role, "->", type(part).to_dict(part))
    print('-'*80)

user -> {'text': 'apnader menu te ki ki item ase?'}
--------------------------------------------------------------------------------
model -> {'function_call': {'name': 'check_menu', 'args': {}}}
--------------------------------------------------------------------------------
user -> {'function_response': {'name': 'check_menu', 'response': {'result': 'The available menu items are: juice, coffee, rice, biriyani, chicken, beef.'}}}
--------------------------------------------------------------------------------
model -> {'text': 'Apnader menu te juice, coffee, rice, biriyani, chicken, beef ache. \n'}
--------------------------------------------------------------------------------
user -> {'text': 'biriyani order krbo'}
--------------------------------------------------------------------------------
model -> {'text': 'Okay, biriyani order korte chachen.  Kichu variation ache ki? Jeno, chicken biriyani, beef biriyani, veg biriyani etc? \n'}
-------------------------------------------------